In [10]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import json
import math
from tqdm import notebook
import time
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
import csv

### 데이터 수집

In [11]:
game_list = pd.read_excel('./game_1.xlsx', usecols=[1, 2])
game_list

,appid,name
0,216938,Pieterw test app76 ( 216938 )
1,660010,test2
2,660130,test3
3,1610510,放置勇者：远征/Idle Heroes:Odyssey-弗雷斯特Forrester
4,1610530,The Annoying Game
...,...,...
9873,1365720,Kujlevka
9874,1365750,东方栖霞园 ~ Blue devil in the Belvedere.
9875,1365760,Defold
9876,1365780,Tales Of Chandar


In [12]:
col_list = [
'short_description',
'price',
'categories',
'genres',
'recommendations',
'release_date',
'developers',
'publishers',
'image'
]

In [13]:
for col in col_list : 
    game_list[f'{col}'] = None

In [14]:
game_list = game_list.set_index('appid')

In [15]:
game_list

,name,short_description,price,categories,genres,recommendations,release_date,developers,publishers,image
appid,,,,,,,,,,
216938,Pieterw test app76 ( 216938 ),None,None,None,None,None,None,None,None,None
660010,test2,None,None,None,None,None,None,None,None,None
660130,test3,None,None,None,None,None,None,None,None,None
1610510,放置勇者：远征/Idle Heroes:Odyssey-弗雷斯特Forrester,None,None,None,None,None,None,None,None,None
1610530,The Annoying Game,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
1365720,Kujlevka,None,None,None,None,None,None,None,None,None
1365750,东方栖霞园 ~ Blue devil in the Belvedere.,None,None,None,None,None,None,None,None,None
1365760,Defold,None,None,None,None,None,None,None,None,None


In [16]:
test = game_list.iloc[:30]

In [17]:
test

,name,short_description,price,categories,genres,recommendations,release_date,developers,publishers,image
appid,,,,,,,,,,
216938,Pieterw test app76 ( 216938 ),None,None,None,None,None,None,None,None,None
660010,test2,None,None,None,None,None,None,None,None,None
660130,test3,None,None,None,None,None,None,None,None,None
1610510,放置勇者：远征/Idle Heroes:Odyssey-弗雷斯特Forrester,None,None,None,None,None,None,None,None,None
1610530,The Annoying Game,None,None,None,None,None,None,None,None,None
1610540,Train Station Renovation - Germany DLC,None,None,None,None,None,None,None,None,None
1610550,Hunt: Showdown - The Committed,None,None,None,None,None,None,None,None,None
1610580,Supporter Bonus DLC,None,None,None,None,None,None,None,None,None
1610590,PrisonShow Playtest,None,None,None,None,None,None,None,None,None


# 1


In [18]:
# 데이터가 없는 오류를 가진 appid 
no_data_appid = []

appList = []
index = 0

for appid in notebook.tqdm(test.index) :
    time.sleep(1.5)
    
    try:
        response = requests.get(f'http://store.steampowered.com/api/appdetails?appids={appid}&l=koreana').json()
    except:
        continue
        
    game = response[f'{appid}']
    
    # 1. success == False인 경우 제외
    if(game['success'] == False): 
        continue
    
    # 2. 원하는 data가 없는 경우 제외
    data = game['data']
    try:
        genres = data['genres'] # 변형
        categories = data['categories'] # 변형
        descriptions = data['short_description']
        price = data['price_overview']['final']
        recommendations = data['recommendations']['total']
        publishers = data['publishers']
        developers = data['developers']
        release = data['release_date']['data']
        image = data['header_image']
            
    except:
        no_data_appid.append(appid)
        continue
        
    # 3. 데이터 저장
    
    genre_list = []
    category_list = []
    
    for g in genres :
        genre_list.append(g['description'])

    for c in categories :
        category_list.append(c['description'])
    
    data_list = [descriptions, price, category_list, genre_list, recommendations, publishers, developers, release, image]
    print("데이터 저장 ")
    print(data_list)
    # 최종적으로 반영
    test.loc[appid, col_list] = data_list


  0%|          | 0/30 [00:00<?, ?it/s]

In [19]:
test

,name,short_description,price,categories,genres,recommendations,release_date,developers,publishers,image
appid,,,,,,,,,,
216938,Pieterw test app76 ( 216938 ),None,None,None,None,None,None,None,None,None
660010,test2,None,None,None,None,None,None,None,None,None
660130,test3,None,None,None,None,None,None,None,None,None
1610510,放置勇者：远征/Idle Heroes:Odyssey-弗雷斯特Forrester,None,None,None,None,None,None,None,None,None
1610530,The Annoying Game,None,None,None,None,None,None,None,None,None
1610540,Train Station Renovation - Germany DLC,None,None,None,None,None,None,None,None,None
1610550,Hunt: Showdown - The Committed,None,None,None,None,None,None,None,None,None
1610580,Supporter Bonus DLC,None,None,None,None,None,None,None,None,None
1610590,PrisonShow Playtest,None,None,None,None,None,None,None,None,None


In [20]:
test = game_list.iloc[:30]

# 2


In [21]:
# 데이터가 없는 오류를 가진 appid 
no_data_appid = []

appList = []
index = 0

for appid in notebook.tqdm(test.index) :
    time.sleep(1.5)

    try:
        response = requests.get(f'http://store.steampowered.com/api/appdetails?appids={appid}&l=koreana').json()
    except:
        continue
        
    game = response[f'{appid}']
    
    # 1. success == False인 경우 제외
    if(game['success'] == False): 
        continue


    # 2. 원하는 data가 없는 경우 제외
    data = game['data']
    
    try:
        price = data['price_overview']['final']
        test.loc[appid, 'price'] = price
    except:
        no_data_appid.append(appid)
        continue

    try:
        genres = data['genres'] # 변형
        genre_list = []

        for g in genres :
            genre_list.append(g['description'])

        test.loc[appid, 'genres'] = genre_list
    except:
        no_data_appid.append(appid)
        continue

    try:
        categories = data['categories'] # 변형
        category_list = []
        for c in categories :
            category_list.append(c['description'])


        test.loc[appid, 'categories'] = category_list

    except:
        no_data_appid.append(appid)
        continue

    try:
        descriptions = data['short_description']
        test.loc[appid, 'short_description'] = descriptions
    except:
        no_data_appid.append(appid)

    try:
        recommendations = data['recommendations']['total']
        test.loc[appid, 'recommendations'] = recommendations
    except:
        no_data_appid.append(appid)

    try:
        publishers = data['publishers']
        test.loc[appid, 'publishers'] = publishers
    except:
        no_data_appid.append(appid)


    try:
        developers = data['developers']
        test.loc[appid, 'developers'] = developers
    except:
        no_data_appid.append(appid)


    try:
        release = data['release_date']['date']
        test.loc[appid, 'release_date'] = release
    except:
        no_data_appid.append(appid)

    try:
        image = data['header_image']
        test.loc[appid, 'image'] = image
    except:
        no_data_appid.append(appid)



  0%|          | 0/30 [00:00<?, ?it/s]

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_25840\4115049265.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[appid, 'price'] = price
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_25840\4115049265.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[appid, 'genres'] = genre_list
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_25840\4115049265.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[appid, 'categori

In [22]:
test

,name,short_description,price,categories,genres,recommendations,release_date,developers,publishers,image
appid,,,,,,,,,,
216938,Pieterw test app76 ( 216938 ),None,None,None,None,None,None,None,None,None
660010,test2,None,None,None,None,None,None,None,None,None
660130,test3,None,None,None,None,None,None,None,None,None
1610510,放置勇者：远征/Idle Heroes:Odyssey-弗雷斯特Forrester,None,None,None,None,None,None,None,None,None
1610530,The Annoying Game,"Run, jump, die, repeat. Help Teddy on his jour...",440000,[싱글 플레이어],"[캐주얼, 인디]",None,2021년 7월 14일,"[Michal Ambrozej, The Piano Chap]","[Michal Ambrozej, The Piano Chap]",https://cdn.akamai.steamstatic.com/steam/apps/...
1610540,Train Station Renovation - Germany DLC,Continue your journey as a renovation company ...,1350000,"[싱글 플레이어, 다운로드 가능한 콘텐츠, Steam 도전 과제, Steam Cloud]","[어드벤처, 캐주얼, 인디, RPG, 시뮬레이션]",None,2022년 4월 5일,[Live Motion Games],[Live Motion Games],https://cdn.akamai.steamstatic.com/steam/apps/...
1610550,Hunt: Showdown - The Committed,This DLC contains one Legendary Hunter (Monroe...,750000,"[멀티플레이어, PvP, 온라인 PvP, 협동, 온라인 협동, 다운로드 가능한 콘텐...",[액션],187,2021년 5월 26일,[Crytek],[],https://cdn.akamai.steamstatic.com/steam/apps/...
1610580,Supporter Bonus DLC,This DLC unlocks a new option in the settings ...,230000,"[싱글 플레이어, 다운로드 가능한 콘텐츠, Steam 도전 과제, Steam Cloud]",[인디],None,2021년 4월 29일,[NanningsGames],[NanningsGames],https://cdn.akamai.steamstatic.com/steam/apps/...
1610590,PrisonShow Playtest,None,None,None,None,None,None,None,None,None


In [23]:
test = game_list.iloc[:30]

# 하나만 테스트

In [24]:
############## 하나만 테스트
# 데이터가 없는 오류를 가진 appid 
no_data_appid = []
appid=1610510
try:
    response = requests.get(f'http://store.steampowered.com/api/appdetails?appids=1610510&l=koreana').json()

except:
    print("False")


game = response[f'{appid}']

# 1. success == False인 경우 제외
if(game['success'] == False): 
    print("False")

# 2. 원하는 data가 없는 경우 제외
data = game['data']

try:
    genres = data['genres'] # 변형
    print(genres)
    genre_list = []

    for g in genres :
        genre_list.append(g['description'])

    test.loc[appid, 'genres'] = genre_list
except:
    no_data_appid.append(appid)

# try:
categories = data['categories'] # 변형
print(categories)
category_list = []
for c in categories :
    category_list.append(c['description'])


test.loc[appid, 'categories'] = category_list

# except:
#     no_data_appid.append(appid)

try:
    descriptions = data['short_description']
    print(descriptions)
    test.loc[appid, 'short_description'] = descriptions
except:
    no_data_appid.append(appid)

try:
    price = data['price_overview']['final']
    print(price)
    test.loc[appid, 'price'] = price
except:
    no_data_appid.append(appid)

try:
    recommendations = data['recommendations']['total']
    print(recommendations)
    test.loc[appid, 'recommendations'] = recommendations
except:
    no_data_appid.append(appid)

try:
    publishers = data['publishers']
    print(publishers)
    test.loc[appid, 'publishers'] = publishers
except:
    no_data_appid.append(appid)
    

try:
    developers = data['developers']
    print(developers)
    test.loc[appid, 'developers'] = developers
except:
    no_data_appid.append(appid)
    

try:
    release = data['release_date']['date']
    print(release)
    test.loc[appid, 'release_date'] = release
except:
    no_data_appid.append(appid)

try:
    image = data['header_image']
    print(image)
    test.loc[appid, 'image'] = image
except:
    no_data_appid.append(appid)


[{'id': '25', 'description': '어드벤처'}, {'id': '4', 'description': '캐주얼'}, {'id': '23', 'description': '인디'}, {'id': '3', 'description': 'RPG'}, {'id': '28', 'description': '시뮬레이션'}, {'id': '2', 'description': '전략'}]
[{'id': 2, 'description': '싱글 플레이어'}, {'id': 21, 'description': '다운로드 가능한 콘텐츠'}, {'id': 23, 'description': 'Steam Cloud'}]
Purchase to acquire the dual action hunter hero:Forrester, and three of Forrester's treasures: Elf Bow, Elf Leather Armor, and Agile Dagger.
['OnigiriStudio']
['OnigiriStudio']
2021년 5월 14일
https://cdn.akamai.steamstatic.com/steam/apps/1610510/header.jpg?t=1620984088


C:\Users\SSAFY\AppData\Local\Temp\ipykernel_25840\1430992953.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[appid, 'genres'] = genre_list
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_25840\1430992953.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[appid, 'categories'] = category_list
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_25840\1430992953.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[app

In [25]:
test

,name,short_description,price,categories,genres,recommendations,release_date,developers,publishers,image
appid,,,,,,,,,,
216938,Pieterw test app76 ( 216938 ),None,None,None,None,None,None,None,None,None
660010,test2,None,None,None,None,None,None,None,None,None
660130,test3,None,None,None,None,None,None,None,None,None
1610510,放置勇者：远征/Idle Heroes:Odyssey-弗雷斯特Forrester,Purchase to acquire the dual action hunter her...,None,"[싱글 플레이어, 다운로드 가능한 콘텐츠, Steam Cloud]","[어드벤처, 캐주얼, 인디, RPG, 시뮬레이션, 전략]",None,2021년 5월 14일,[OnigiriStudio],[OnigiriStudio],https://cdn.akamai.steamstatic.com/steam/apps/...
1610530,The Annoying Game,"Run, jump, die, repeat. Help Teddy on his jour...",440000,[싱글 플레이어],"[캐주얼, 인디]",None,2021년 7월 14일,"[Michal Ambrozej, The Piano Chap]","[Michal Ambrozej, The Piano Chap]",https://cdn.akamai.steamstatic.com/steam/apps/...
1610540,Train Station Renovation - Germany DLC,Continue your journey as a renovation company ...,1350000,"[싱글 플레이어, 다운로드 가능한 콘텐츠, Steam 도전 과제, Steam Cloud]","[어드벤처, 캐주얼, 인디, RPG, 시뮬레이션]",None,2022년 4월 5일,[Live Motion Games],[Live Motion Games],https://cdn.akamai.steamstatic.com/steam/apps/...
1610550,Hunt: Showdown - The Committed,This DLC contains one Legendary Hunter (Monroe...,750000,"[멀티플레이어, PvP, 온라인 PvP, 협동, 온라인 협동, 다운로드 가능한 콘텐...",[액션],187,2021년 5월 26일,[Crytek],[],https://cdn.akamai.steamstatic.com/steam/apps/...
1610580,Supporter Bonus DLC,This DLC unlocks a new option in the settings ...,230000,"[싱글 플레이어, 다운로드 가능한 콘텐츠, Steam 도전 과제, Steam Cloud]",[인디],None,2021년 4월 29일,[NanningsGames],[NanningsGames],https://cdn.akamai.steamstatic.com/steam/apps/...
1610590,PrisonShow Playtest,None,None,None,None,None,None,None,None,None


### ------------
